In [4]:
import pandas as pd
import numpy as np 

path = '/Users/miao.shang/Downloads/some score examples.csv'
df = pd.read_csv(path)

In [5]:
print(df.head())

                 Record Id Score updated date Overall recommendation  \
0  zcrm_699287000002448175         2025-12-11                 ignore   
1  zcrm_699287000002469899         2025-12-11                 review   
2  zcrm_699287000002502050         2025-12-11                 ignore   
3  zcrm_699287000002517349         2025-12-11                 review   
4  zcrm_699287000002539665         2025-12-11                 review   

          Lead Owner            Created By.id    Created By  \
0       Maike Brahms  zcrm_699287000000512001  Daniel Lykke   
1        Mimi Jamily  zcrm_699287000000512001  Daniel Lykke   
2     Koen Besseling  zcrm_699287000001851017   Omar Walker   
3        System User  zcrm_699287000000512001  Daniel Lykke   
4  Estefania Distola  zcrm_699287000001856001  Nacho Merino   

                                Instagram link  Instagram Handle  SoMe score  \
0        https://www.instagram.com/renevogel26       renevogel26        2.57   
1              https://www.i

In [6]:
SOME_REF = {
    'pf': 1.2544,             # posting_frequency_c reference
    'er': 9.0481,             # engagement_rate reference
    'fc': (961, 467930),      # follower_count_c low/high refs
}
SOME_W = {
    'pf': 1.76,
    'er': 4.48,
    'fc': 3.76,
}

In [7]:
def compute_fc_score(value, low_ref, high_ref, weight):
    if pd.isna(value):
        return 0.0
    if value <= low_ref:
        return 0.0
    if value >= high_ref:
        return float(weight)
    return float((value - low_ref) / (high_ref - low_ref) * weight)

def compute_score(value, reference, weight):
    if pd.isna(value):
        return 0.0
    return float(weight if value >= reference else (value / reference) * weight)

def ensure_cols(df, cols, fill=0.0):
    """Make sure required columns exist; if missing, create them filled with 'fill'."""
    for c in cols:
        if c not in df.columns:
            df[c] = fill


In [11]:
def add_some_category_only(df: pd.DataFrame) -> pd.DataFrame:
    """
    Keep ALL existing columns, add TWO new columns:
    - total_score
    - category_some_2_0
    """
    out = df.copy()

    ensure_cols(out, ['Posting frequency', 'Engagement Rate', 'Follower count'], fill=0.0)

    def assign_category(row):
        pf_score = compute_score(row['Posting frequency'], SOME_REF['pf'], SOME_W['pf'])
        er_score = compute_score(row['Engagement Rate'], SOME_REF['er'], SOME_W['er'])
        fc_score = compute_fc_score(
            row['Follower count'],
            SOME_REF['fc'][0],
            SOME_REF['fc'][1],
            SOME_W['fc']
        )
        total_score = pf_score + er_score + fc_score

        if pf_score >= 1. and er_score >= 1. and fc_score >= 1. and total_score >= 6:
            category = 'ignore'
        elif fc_score >= 0.5 and total_score >= 5:
            category = 'launch'
        elif 1.22 <= total_score < 5.2 and er_score >= 0.7 and fc_score >= 2.86:
            category = 'launch'
        elif er_score < 0.7 and total_score < 1.22:
            category = 'ignore'
        elif 3 <= total_score < 4.66 and 0.35 <= pf_score <= 1 and er_score >= 0.87:
            category = 'launch'
        elif 1.76 < total_score < 3.02:
            category = 'ignore'
        else:
            category = 'review'

        return pd.Series({
            "total_score": total_score,
            "category_some_2_0": category
        })

    out[["total_score", "category_some_2_0"]] = out.apply(assign_category, axis=1)
    return out


In [15]:
out = add_some_category_only(df)

print(out.head(3))


                 Record Id Score updated date Overall recommendation  \
0  zcrm_699287000002448175         2025-12-11                 ignore   
1  zcrm_699287000002469899         2025-12-11                 review   
2  zcrm_699287000002502050         2025-12-11                 ignore   

       Lead Owner            Created By.id    Created By  \
0    Maike Brahms  zcrm_699287000000512001  Daniel Lykke   
1     Mimi Jamily  zcrm_699287000000512001  Daniel Lykke   
2  Koen Besseling  zcrm_699287000001851017   Omar Walker   

                                Instagram link  Instagram Handle  SoMe score  \
0        https://www.instagram.com/renevogel26       renevogel26        2.57   
1              https://www.instagram.com/_mssn             _mssn        4.26   
2  https://www.instagram.com/gemma_alicia_fit/  gemma_alicia_fit        1.77   

  Main social media  ...  Follower count  Posts last 30 days  \
0         Instagram  ...         17311.0                 3.0   
1         Instagram  

In [16]:
out.to_csv("out.csv", index=False)
